In [ ]:
##------------------detect tremors-----------------##
import obspy
from obspy.clients.fdsn import Client
client = Client("USGS")
from obspy import UTCDateTime
import numpy as np
from obspy.geodetics import locations2degrees
from obspy.geodetics import gps2dist_azimuth
from obspy.geodetics import degrees2kilometers
from obspy import read

import scipy
import scipy.signal as scisignal
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from obspy.taup import TauPyModel
model = TauPyModel(model="iasp91")

from PIL import Image
from IPython.display import Image
from obspy.signal.trigger import classic_sta_lta
from obspy.signal.trigger import plot_trigger

import os
# import geopy.distance
import glob
import obspy.signal.filter
import pandas as pd
import glob
import csv
import pickle
import datetime
import copy
import scipy
import statistics
from scipy.cluster.hierarchy import single, fcluster,dendrogram
from collections import Counter

In [ ]:
#-----------load obs data----------------##
obs = pd.read_csv('Seismometer_station',sep ='   ',header=None)

In [ ]:
##------two earthquake catalogs---------##
eqcat = pd.read_csv('located_catalog_scak400km',sep =' ')
equsgs = pd.read_csv('usgs_20180501_20190831.csv')

datetime_eq = [datetime.datetime.strptime(itime,'%Y-%m-%dT%H:%M:%S.%f') for itime in eqcat['Time']]
datetime_usgs = [datetime.datetime.strptime(itime,'%Y-%m-%dT%H:%M:%S.%fZ') for itime in equsgs['time']]

In [ ]:
Direc = '2019-**-**'
datetime0 = datetime.datetime.strptime(Direc,'%Y-%m-%d')
l1 = 2; l2 = 8; h1 = 10; h2 = 16;
## set up a day
for iday in range(***):
    datetime1 = datetime0 + datetime.timedelta(days = iday)
    Direc = datetime.datetime.strftime(datetime1,'%Y-%m-%d')
    print(Direc)
##----------load cross-correlations--------------------##
    with open(Direc + '/station_name.npy','rb') as f:
        sta_name2 = np.load(f)        
    with open(Direc + '/cc_day_lf_'+str(l1)+'-'+str(l2)+'_'+str(h1)+'-'+str(h2)+'.pkl','rb') as f1:
        cc_day_lf = pickle.load(f1)
    with open(Direc + '/cc_day_hf_'+str(l1)+'-'+str(l2)+'_'+str(h1)+'-'+str(h2)+'.pkl',"rb") as f1:
        cc_day_hf = pickle.load(f1)
    with open(Direc + '/cc_day_rf_EP22_'+str(l1)+'-'+str(l2)+'.pkl',"rb") as f1:
        cc_day_rf1 = pickle.load(f1)
    with open(Direc + '/cc_day_rf_WD53_'+str(l1)+'-'+str(l2)+'.pkl',"rb") as f1:
        cc_day_rf2 = pickle.load(f1)
    with open(Direc + '/cc_day_rf_LT20_'+str(l1)+'-'+str(l2)+'.pkl',"rb") as f1:
        cc_day_rf3 = pickle.load(f1)
    with open(Direc + '/cc_day_rf_WD67_'+str(l1)+'-'+str(l2)+'.pkl',"rb") as f1:
        cc_day_rf4 = pickle.load(f1)
        
##-----------calculate low-frequency tremor pairs---------------##
##-----------At least three pairs with distance between stations and mean point less than 80km and CC>0.68-------##
## Hierarchical cluster method        
    tremor_day = dict.fromkeys(range(len(cc_day_lf)))
    for istep in range(len(cc_day_lf)):
        tremor_pair = []        
        cc_day_pdist = []
        for ista1 in range(len(sta_name2)):
            for ista2 in range(ista1+1,len(sta_name2)):
                cc_day_pdist.append(abs(1/(cc_day_lf[istep][ista1][ista2]+1e-10)))
        Z = single(cc_day_pdist)
        clusters = fcluster(Z, 1/0.68, criterion='distance')
        dict1 = dict(Counter(clusters))
        counter_output = []
        for key in dict1:
            if dict1[key] >= 3:
                counter_output.append(key)

        for ic in range(len(counter_output)):
            major_cluster = np.where(clusters == counter_output[ic])[0]
#             print('istep:'+str(istep)+' cluster:'+str(major_cluster))
            if len(major_cluster)<=6:
                lat = []; lon = []
                for i in range(len(major_cluster)):
                    iw1 = np.where(obs.iloc[:,1]==sta_name2[major_cluster[i]])
                    lat.append(obs.iloc[iw1[0][0],2])
                    lon.append(obs.iloc[iw1[0][0],3])
                latc = np.mean(lat)
                lonc = np.mean(lon)
                for i in range(len(lat)):
                    degkm = degrees2kilometers(locations2degrees(lat[i],lon[i],latc,lonc))
                    if degkm>80:
                        major_cluster[i] = -1
                major_cluster2 = major_cluster[major_cluster>=0]
                if len(major_cluster2)>=3:
                    tremor_pair.append(major_cluster2)
        tremor_day[istep] = tremor_pair
    for istep in range(len(cc_day_lf)):
        if len(tremor_day[istep]) == 0:
            del tremor_day[istep]
    print(tremor_day.keys())
    
##-----------calculate high-frequency earthquake pairs---------------##
##-----------At least six pairs over 0.65-------##    
    ## Earthquake detections
    eq_day = dict.fromkeys(range(len(cc_day_hf)))
    for istep in range(len(cc_day_hf)):
        eq_pair = []        
        cc_day_pdist = []
        for ista1 in range(len(sta_name2)):
            for ista2 in range(ista1+1,len(sta_name2)):
                if np.isnan(cc_day_hf[istep][ista1][ista2]):
                    cc_day_pdist.append(1/1e-10)
                else:
                    cc_day_pdist.append(abs(1/(cc_day_hf[istep][ista1][ista2]+1e-10)))
        Z = single(cc_day_pdist)
        clusters = fcluster(Z, 1/0.65, criterion='distance')
        dict1 = dict(Counter(clusters))
        counter_output = []
        for key in dict1:
            if dict1[key] >= 6:
                counter_output.append(key)

        for ic in range(len(counter_output)):
            major_cluster = np.where(clusters == counter_output[ic])[0]
            eq_pair.append(major_cluster)
        eq_day[istep] = eq_pair
    for istep in range(len(cc_day_hf)):
        if len(eq_day[istep]) == 0:
            del eq_day[istep]
    print(eq_day.keys())    
    
##-----------calculate regioanl earthquake pairs---------------##
##----------- At least three station pairs over 0.65-----------##
    numsta_rf = 3
    threshold_rf = 0.65
    rf_day = dict.fromkeys(range(len(cc_day_rf1)))
    for istep in range(len(cc_day_rf1)):
        cc1 = cc_day_rf1[istep]
        ii1 = np.where(cc1>=threshold_rf)
        cc2 = cc_day_rf2[istep]
        ii2 = np.where(cc2>=threshold_rf)
        cc3 = cc_day_rf3[istep]
        ii3 = np.where(cc3>=threshold_rf)
        cc4 = cc_day_rf4[istep]
        ii4 = np.where(cc4>=threshold_rf)        
        if (len(ii1[0])>=numsta_rf or len(ii2[0])>=numsta_rf or len(ii3[0])>=numsta_rf or len(ii4[0])>=numsta_rf):
            rf_day[istep] = max(len(ii1[0]),len(ii2[0]),len(ii3[0]),len(ii4[0]))
        else:
            del rf_day[istep]
    print(rf_day.keys())

##-------Remove local and regional earthquakes from the tremor pairs-----------##
    tremor_day2 = copy.deepcopy(tremor_day)
    for istep in tremor_day.keys():
        if istep in eq_day.keys():
            del tremor_day2[istep]
        if istep in rf_day.keys() and istep not in eq_day.keys():
            del tremor_day2[istep]
    try:
        del tremor_day2[0]
    except:
        print('No zero')
    print(tremor_day2.keys())

##---------Remove earthquake catalogs from tremor pairs-------------##
    tremor_day3 = copy.deepcopy(tremor_day2)
    for istep in tremor_day2.keys():
        t1 = 1.5*istep-1.5
        t2 = t1+4.5
        datetime2 = datetime1 + datetime.timedelta(minutes = t1)
        datetime3 = datetime1 + datetime.timedelta(minutes = t2)
        for itime in datetime_eq:
            if itime>=datetime2 and itime<=datetime3:
                print('wxz:'+str(itime))
                try: 
                    del tremor_day3[istep]
                except:
                    print('Delete by XZW already!')

        for itime in datetime_usgs:
            if itime>=datetime2 and itime<=datetime3:
                print('usgs:'+str(itime))                      
                try: 
                    del tremor_day3[istep]
                except:
                    print('Delete already!')    
    print("tremor3:")
    print(tremor_day3.keys()) 

##---------Delete long distance stations over 120 km (replace them with 0)-------##
    dist = 120
    tremor_day4 = copy.deepcopy(tremor_day3)
    for item in tremor_day3.keys():
        for item2 in range(len(tremor_day3[item])):
            lat = []; lon = [];
            for ista in range(len(tremor_day3[item][item2])):
                iw1 = np.where(obs.iloc[:,1]==sta_name2[tremor_day3[item][item2][ista]])
                lat.append(obs.iloc[iw1[0][0],2])
                lon.append(obs.iloc[iw1[0][0],3])
            list1 = np.arange(0,len(lat))
            list2 = np.arange(0,len(lon))
            result = [(i,j) for i in list1 for j in list2 if i != j]
            for iw in range(len(result)):
                degkm = degrees2kilometers(locations2degrees(lat[result[iw][0]],lon[result[iw][0]],lat[result[iw][1]],lon[result[iw][1]]))
                if degkm >= dist:
                    tremor_day4[item][item2] = 0
                    break
    print("tremor4:")
    print(tremor_day4)
##-------- Remove 0 detections in tremor pairs---------------##
    tremor_day5 = copy.deepcopy(tremor_day4)
    for item in tremor_day4.keys():
        if len(tremor_day4[item])<2:
            if np.all(tremor_day4[item][0] == 0):
                del tremor_day5[item]
        else:
            item2 = []
            for list_i, list_element in enumerate(tremor_day4[item]):
                if np.all(list_element != 0):
                    item2.append(list_element)
            tremor_day5[item] = item2
            if len(tremor_day5[item]) == 0:
                del tremor_day5[item]        
    print("tremor5:")
    print(tremor_day5)      
    
    # save data
    with open(Direc + '/tremor_day_180s_lfhf_'+str(l1)+'-'+str(l2)+'_'+str(h1)+'-'+str(h2)+'_hierarchy2_'+str(dist)+'km_40shift.pkl',"wb") as f1:
        pickle.dump(tremor_day5,f1)      